# Select the most difficult images to label next

*This notebook shows you how to plug your ML model in Encord Active to rank the images according to an acquisition function.
We assume that you already have installed encord-active in your local environment, if not please visit [here](https://docs.encord.com/active/docs/installation).*

### The  Motivation

Let's say you have an image dataset consist of different airplanes. You want to train an object detector model to get the bounding boxes of the planes and later crop those regions to create a collage of airplanes pictures.
[Here](https://storage.googleapis.com/encord-active-notebook-demos/airplanes-active-learning-demo.zip), we have an Encord Active project like this. Download the zipped folder, extract it, and run encord active in the project folder or its parent folder to visualize this project.

To start the Encord-Active app, run the following command:
`encord-active visualize`

You can visualize the images according to certain image-level metrics in the Data Quality -> Explorer page.

There are 799 images in the dataset; however, you do not want to label all of them, which is quite cumbersome. You only want to label the first 50 and you want to select the best 50 images that will give you the best outcome in terms of model performance.

In an active learning workflow, you generally have a model trained on a few initial labeled samples. In this notebook, for the simplicity, we will use a HuggingFace YOLOS-tiny model that is trained on COCO dataset, in which there is an `airplane` class! So, we will use the YOLOS-tiny model's output for the `airplane` class to rank the images.

As an acquisition function, you will use Encord Active's `AverageFrameScore`. This acquisition function gets the average confidence scores of the predictions in each image, if there is no prediction, it assigns zero. You think that in all of your images there is at least one airplane, so this acquisition function is very suitable for your task. If the score is low, model is having difficulty in detecting airplanes in those images, so you may want to collect these samples to label next.

Let's start, you will see how easy to do that with Encord Active!

## 1. Install necessary libraries

In [ ]:
! pip install transformers
! pip install Pillow

## 2. Import the necessary libraries

In [ ]:
from pathlib import Path
from typing import List
from PIL import Image
from transformers import YolosImageProcessor, YolosForObjectDetection
import torch
from encord_active.lib.metrics.acquisition_metrics.object_level_acquisition_functions import (
    AverageFrameScore,
    BaseBoundingBoxModelWrapper,
    BoundingBoxPrediction,
)
from encord_active.lib.metrics.execute import execute_metrics
from encord_active.lib.project.project_file_structure import ProjectFileStructure
from encord_active.lib.metrics.metadata import fetch_metrics_meta
from encord_active.lib.metrics.io import  get_metric_metadata
from encord_active.lib.metrics.metadata import update_metrics_meta

## 3. Implement a BaseBoundingBoxModelWrapper class

This class will inherit the `BaseBoundingBoxModelWrapper` class and only  implement the `predict_boundingboxes()` method, where a Pillow's Image is taken as an argument and list of bounding boxes will be returned.


In [ ]:
class HuggingFaceYolosTinyModel(BaseBoundingBoxModelWrapper):
    """
    For more information on how to use this model, check here: https://huggingface.co/hustvl/yolos-tiny
    """
    def __init__(self):
        self.processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")
        self.model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')

    @torch.no_grad()
    def predict_boundingboxes(self, image: Image)-> List[BoundingBoxPrediction]:
        if image.mode != 'RGB':
            image = image.convert('RGB')

        data = self.processor(images=image, return_tensors="pt")
        outputs = self.model(**data)

        target_sizes = torch.tensor([image.size[::-1]])
        results = self.processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.5)[0]

        output: List[BoundingBoxPrediction] = []
        for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
            if label == 5: # id 5 corresponds to 'airplane' class in COCO Dataset
                box = box.numpy()
                output.append(BoundingBoxPrediction(x=box[0], y=box[1], w=box[2]-box[0], h=box[3]-box[1], confidence=score.item()))

        return output

## 4. Run the acquisition metric

In [ ]:
project_fs = ProjectFileStructure(Path("/path/to/the/project"))

# Create the acquisition metric
acq_func = AverageFrameScore(HuggingFaceYolosTinyModel())

# Run the acquisition metric
execute_metrics([acq_func], data_dir=project_fs.project_dir, use_cache_only=True)

# Update the metric files
metrics_meta = fetch_metrics_meta(project_fs)
metrics_meta[acq_func.metadata.title]= get_metric_metadata(acq_func)
update_metrics_meta(project_fs, metrics_meta)

project_fs.db.unlink(missing_ok=True)

## 5. Next steps

Now, we have run our acquisition function on all samples. When you open the project from the start, you will see this new acquisition metric in the Data Quality Explorer page. Click the arrow in the metrics dropdown list and select 'Average Frame Score'. The images that have a score of zero represents that your current model is missing the airplanes in these examples and low scores mean that model is not confident in its predictions, so they should have higher priority when labelling. You have several options now. You can select the first N samples and:

1. You can select the first N samples and create a new project on Encord Annotate platform to label.
2. Export them using the Actions tab and use them in your own annotation tool.